# Análisis Exploratorio y Limpieza de Datos

## 1. Configuración de Entorno y Carga de Datos

Esta sección inicializa el entorno de trabajo, importando las librerías necesarias y configurando las rutas de los archivos. Se cargan los datos crudos desde un archivo CSV para comenzar el análisis.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import sys
import os

# Add src directory to path to import our custom module
# This makes the notebook runnable from the 'notebooks' directory
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
from data_cleaner import run_cleaning_pipeline

# Set plotting style
sns.set_theme(style="whitegrid")

# Define relative file paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
input_file_path = os.path.join(project_root, 'data', 'rentabilidad_productos.csv')
output_file_path = os.path.join(project_root, 'data', 'rentabilidad_productos_limpio.parquet')

Para asegurar que el notebook no falle si el archivo de datos no se encuentra, la carga se realiza dentro de un bloque `try-except`. La variable `df_raw` se inicializa como `None` para manejar este caso de forma controlada.

In [ ]:
# Initialize df_raw to None to handle potential FileNotFoundError
df_raw = None
try:
    df_raw = pd.read_csv(input_file_path)
    print(f"DataFrame crudo cargado con {df_raw.shape[0]} filas y {df_raw.shape[1]} columnas.")
    display(df_raw.head())
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta esperada: {input_file_path}")
    print("Por favor, asegúrese de que 'rentabilidad_productos.csv' exista en el directorio 'data'.")

## 2. Análisis Exploratorio de Datos Crudos (EDA)

Antes de cualquier modificación, se realiza un análisis de los datos originales. El objetivo es entender su estructura, identificar problemas de calidad como valores nulos, duplicados o tipos de datos incorrectos.

### 2.1. Tipos de Datos y Resumen General

El primer paso es obtener una visión general de la estructura del DataFrame. Se utiliza `info()` para revisar las columnas, el número de registros no nulos y, fundamentalmente, sus tipos de datos (`Dtype`), lo que permite detectar problemas obvios como números almacenados como texto.

In [ ]:
df_raw.info()

### 2.2. Análisis de Valores Nulos

Se cuantifican los valores nulos para cada columna y se visualiza su distribución con la librería `missingno`. Esto ayuda a identificar si los datos faltantes siguen algún patrón o son aleatorios.

In [ ]:
missing_values = df_raw.isnull().sum()
missing_percentage = (missing_values / len(df_raw)) * 100
missing_df = pd.DataFrame({'count': missing_values, 'percentage': missing_percentage})
missing_df = missing_df[missing_df['count'] > 0].sort_values(by='count', ascending=False)

print(f"Se encontraron {len(missing_df)} columnas con valores nulos en los datos crudos.")
display(missing_df)

msno.matrix(df_raw)
plt.show()

### 2.3. Análisis de Duplicados en Clave Primaria (SKU)

El `sku` debería ser un identificador único para cada producto. Se verifica si existen SKUs duplicados, lo cual indicaría un problema de integridad de datos. Para asegurar un conteo preciso, los nombres de columna se normalizan temporalmente (a minúsculas) antes de la verificación.

In [ ]:
# We need to normalize column names first to access 'sku' reliably
temp_df = df_raw.copy()
temp_df.columns = temp_df.columns.str.strip().str.lower()

sku_counts = temp_df.sku.value_counts()
duplicated_skus = sku_counts[sku_counts > 1]

print(f"Análisis de duplicados de SKU:")
print(f"- Hay {len(sku_counts)} SKUs únicos de un total de {len(temp_df)} registros.")
print(f"- Se encontraron {len(duplicated_skus)} SKUs con registros duplicados.")

## 3. Proceso de Limpieza de Datos

Se invoca el pipeline de limpieza definido en el módulo `data_cleaner.py`. Esta función centralizada aplica todas las reglas de negocio y transformaciones descubiertas en el AED para producir un DataFrame limpio, manteniendo este notebook enfocado en el análisis y no en la implementación detallada.

In [ ]:
df_cleaned = run_cleaning_pipeline(df_raw)

## 4. Verificación y Análisis de Datos Limpios

Una vez limpios los datos, es crucial verificar que el proceso fue exitoso y que los datos resultantes son coherentes. Se revisan los tipos de datos, la ausencia de nulos y la integridad de los valores.

### 4.1. Verificación de Tipos de Datos y Nulos

Se repite la ejecución de `info()` y `isnull().sum()` sobre el DataFrame limpio. El objetivo es confirmar que las columnas numéricas y de fecha tienen los tipos correctos y que los valores nulos han sido tratados según la estrategia definida.

In [ ]:
print("--- Tipos de datos después de la limpieza ---")
df_cleaned.info()

print("\n--- Conteo de Nulos Restantes ---")
print(df_cleaned.isnull().sum())

### 4.2. Verificación de Integridad de Datos (Cálculo de Margen)

Esta es una validación de lógica de negocio: se calcula el margen de forma manual (`precio_venta` - `precio_compra`) y se compara con la columna `margen` original. Para evitar problemas con la precisión de los números de punto flotante, se utiliza una pequeña tolerancia de 0.01 en lugar de una comparación exacta.

In [ ]:
df_cleaned['margen_calculado'] = df_cleaned['precio_venta'] - df_cleaned['precio_compra']
df_cleaned['margen_diferencia'] = (df_cleaned['margen'] - df_cleaned['margen_calculado']).abs()

discrepancies = df_cleaned[df_cleaned['margen_diferencia'] > 0.01] # Use tolerance for float precision

print(f"Se encontraron {len(discrepancies)} filas con una diferencia mayor a 0.01 entre el margen reportado y el calculado.")

### 4.3. Resumen Estadístico del DataFrame Limpio

Se genera un resumen estadístico completo de todas las columnas del DataFrame limpio. Esto proporciona una visión cuantitativa de las distribuciones de datos (media, desviación estándar, cuartiles) y permite identificar posibles anomalías o outliers.

In [ ]:
display(df_cleaned.describe(include='all').T)

### 4.4. Distribución de Variables Numéricas Limpias

Para complementar el resumen estadístico, se visualizan las distribuciones de las columnas numéricas clave mediante histogramas y diagramas de caja (boxplots). Esto permite una inspección visual rápida de la forma de los datos, su simetría y la presencia de valores atípicos.

In [ ]:
# Wrap in a check to ensure df_cleaned exists
if 'df_cleaned' in locals() and df_cleaned is not None:
    for col in ['precio_compra', 'precio_venta', 'margen']:
        fig, axes = plt.subplots(1, 2, figsize=(15, 4))
        fig.suptitle(f"Análisis de la columna numérica limpia: '{col}'", fontsize=16)

        sns.histplot(df_cleaned[col], kde=True, ax=axes[0])
        axes[0].set_title('Distribución (Histograma)')

        sns.boxplot(x=df_cleaned[col], ax=axes[1])
        axes[1].set_title('Diagrama de Caja (Boxplot)')

        # Use a tuple for the 'rect' parameter instead of a list
        plt.tight_layout(rect=(0, 0.03, 1, 0.95))
        plt.show()
else:
    print("El DataFrame 'df_cleaned' no está definido. Saltando la visualización.")

## 5. Almacenamiento del DataFrame Limpio

El paso final es persistir el DataFrame limpio. Antes de guardar, se eliminan las columnas temporales creadas para la verificación (`margen_calculado`, `margen_diferencia`), ya que no son parte del conjunto de datos final. Se elige el formato Parquet por su eficiencia en almacenamiento y velocidad de lectura para futuros análisis.

In [ ]:
# Drop temporary columns used for verification before saving
final_df_to_save = df_cleaned.drop(columns=['margen_calculado', 'margen_diferencia'])

final_df_to_save.to_parquet(output_file_path, index=False, engine='pyarrow')

print(f"DataFrame limpio con {len(final_df_to_save)} filas guardado exitosamente en: {output_file_path}")

## 6. Nota

Durante el proceso de limpieza, se tomaron dos decisiones clave que resultaron en la eliminación de filas del conjunto de datos original:

1.  **Eliminación de SKUs duplicados:** Se eliminaron registros de productos con el mismo `sku`, conservando únicamente el que tenía la `fecha_actualizacion` más reciente. Esta es una estrategia común para obtener la "versión más reciente" de un registro, pero la decisión correcta depende estrictamente del negocio. En otros escenarios, podría ser necesario consolidar o promediar la información de los duplicados, o investigarlos como una anomalía en la fuente de datos.

2.  **Eliminación de Nulos en Columnas Críticas:** Se descartaron filas que contenían valores nulos en columnas consideradas indispensables para el análisis, como `precio_compra`, `precio_venta` o `fecha_actualizacion`. Para este análisis, se asumió que un registro sin esta información carece de valor. Sin embargo, en un entorno de producción, una estrategia alternativa podría ser enviar estos registros a una "cola de errores" para una revisión manual o intentar enriquecer los datos faltantes desde otras fuentes antes de descartarlos.